# OCHRE User Tutorial

* [Set up](#setup)
  * [Installation](#install)
  * [Getting input files](#inputs)
* [Running a basic house](#dwelling)
* [Running a single piece of equipment](#equipment)
  * [Electric Vehicle](#ev)
  * [Water Heater](#water_heater)
* [Running a fleet](#fleet)
  * [EV Fleet](#ev-fleet)
  * [Multiple houses](#house-fleet)
* [Running with external controllers](#control)
  * [HVAC setpoint control](#hvac-control)
  * [EV managed charging](#ev-control)
  * [HPWH CTA-2045 control](#wh-control)
* [Links to other examples](#links)

This tutorial covers the basics of how to install and run OCHRE, and provides
some examples for various use cases. It can be downloaded online
[here](https://github.com/NREL/OCHRE/blob/main/notebook/user_tutorial.ipynb).

The full documentation for OCHRE can be found [here](https://ochre-nrel.readthedocs.io/)


## <a name="setup"></a>Set up


### <a name="install"></a>Installation

OCHRE can be installed using `pip` from the command line with:

```
pip install ochre-nrel
```

Alternatively, you can install a specific branch, for example:

```
pip install git+https://github.com/NREL/OCHRE@dev
```

Note that OCHRE requires Python version >=3.9 and <3.12.

### <a name="inputs"></a>Getting input files

OCHRE Dwelling models often require 3 inputs files:
* An HPXML file with building properties (.xml)
* An occupancy schedule file (.csv)
* A weather file (.epw, or .csv with NSRDB format)

We recommend using [ResStock](https://www.nrel.gov/buildings/resstock.html) (version 3.0+) or
[BEopt](https://www.nrel.gov/buildings/beopt.html) (version 3.0+) to create HPXML and occupancy schedule files.
OCHRE includes some [sample files](https://github.com/NREL/OCHRE/tree/main/ochre/defaults/Input%20Files)
and some code to [download ResStock files](https://github.com/NREL/OCHRE/tree/main/bin/run_multiple.py#L72).

Weather files can be downloaded from [EnergyPlus](https://energyplus.net/weather), [NSRDB](https://nsrdb.nrel.gov/),
[BEopt](https://www.nrel.gov/buildings/beopt.html), or from ResStock's [dataset](https://data.nrel.gov/submissions/156).

See the official documentation for more information on [Generating Input Files](https://ochre-nrel.readthedocs.io/en/latest/InputsAndArguments.html#generating-input-files).

## <a name="dwelling"></a>Running a basic house

For more details, see [bin/run_dwelling.py](https://github.com/NREL/OCHRE/tree/main/bin/run_dwelling.py)

In [ ]:
import os
import datetime as dt
import pandas as pd

from ochre import Dwelling
from ochre.utils import default_input_path  # for using sample files

dwelling_args = {
    "name": "MyHouse",  # simulation name
    
    # Timing parameters
    "start_time": dt.datetime(2018, 1, 1, 0, 0),  # year, month, day, hour, minute
    "time_res": dt.timedelta(minutes=10),         # time resolution of the simulation
    "duration": dt.timedelta(days=1),             # duration of the simulation

    # Input files
    "hpxml_file": os.path.join(default_input_path, "Input Files", "bldg0112631-up11.xml"),
    "hpxml_schedule_file": os.path.join(
        default_input_path, "Input Files", "bldg0112631_schedule.csv"
    ),
    "weather_file": os.path.join(
        default_input_path, "Weather", "USA_CO_Denver.Intl.AP.725650_TMY3.epw"
    ),
    
    # Output parameters
    "verbosity": 4,              # verbosity of time series files (0-9)
    "output_path": os.getcwd(),  # defaults to properties_file path
    
    # Equipment parameters (see bin/run_dwelling.py for more options)
    # "Equipment": {
    #     "PV": {
    #         "capacity": 5,   # in kW
    #         "tilt": 20,      # in degrees
    #         "azimuth": 0,    # in degrees, south=0
    #     }
    # },
}

# Create Dwelling model
dwelling = Dwelling(**dwelling_args)

In [ ]:
# from ochre import Analysis

# Run OCHRE simulation (returns DataFrames of timeseries results and a dictionary of metrics)
df, metrics, hourly = dwelling.simulate()

# Load results from previous run
# df, metrics, hourly = Analysis.load_ochre(dwelling_args["output_path"], dwelling_args["name"])

df.head()

In [ ]:
metrics

In [ ]:
%matplotlib

from ochre import CreateFigures

# Plot results
fig = CreateFigures.plot_power_stack(df)

In [ ]:
fig = CreateFigures.plot_daily_profile(df, 'Total Electric Power (kW)', plot_max=False, plot_min=False)

## <a name="equipment"></a>Running a single equipment model

For more details and examples, see
[bin/run_equipment.py](https://github.com/NREL/OCHRE/blob/main/bin/run_equipment.py)

In [6]:
default_args = {
    "start_time": dt.datetime(2018, 1, 1, 0, 0),  # year, month, day, hour, minute
    "time_res": dt.timedelta(minutes=15),
    "duration": dt.timedelta(days=10),
    "verbosity": 3,
    "save_results": False,  # if True, must specify output_path
    # "output_path": os.getcwd(),
}


### <a name="ev"></a>Running an electric vehicle


In [ ]:
from ochre import ElectricVehicle

equipment_args = {
    # Equipment parameters
    "vehicle_type": "BEV",
    "charging_level": "Level 1",
    "range": 200,
    # Option to specify a file with EV charging events
    # "equipment_event_file": os.path.join(default_input_path, "EV", "PHEV_level_1.csv"),
    **default_args,
}

# Initialize equipment
equipment = ElectricVehicle(**equipment_args)

In [ ]:
# Simulate equipment
df = equipment.simulate()

df.head()


In [ ]:
fig = CreateFigures.plot_daily_profile(df, "EV Electric Power (kW)", plot_max=False, plot_min=False)


In [ ]:
fig = CreateFigures.plot_time_series_detailed((df["EV SOC (-)"],))


### <a name="water_heater"></a>Running a water heater


In [ ]:
import numpy as np
from ochre import ElectricResistanceWaterHeater

# Create water draw schedule
time_res = dt.timedelta(minutes=1)
times = pd.date_range(
    default_args["start_time"],
    default_args["start_time"] + default_args["duration"],
    freq=time_res,
    inclusive="left",
)
water_draw_magnitude = 12  # L/min
withdraw_rate = np.random.choice([0, water_draw_magnitude], p=[0.99, 0.01], size=len(times))
schedule = pd.DataFrame(
    {
        "Water Heating (L/min)": withdraw_rate,
        "Zone Temperature (C)": 20,
        "Mains Temperature (C)": 7,
    },
    index=times,
)

equipment_args = {
    # Equipment parameters
    "Setpoint Temperature (C)": 51,
    "Tank Volume (L)": 250,
    "Tank Height (m)": 1.22,
    "UA (W/K)": 2.17,
    "schedule": schedule,
    **default_args,
    "time_res": time_res,
}

# Initialize equipment
wh = ElectricResistanceWaterHeater(**equipment_args)


In [ ]:
# Run simulation
df = wh.simulate()

# Show results
df.head()


In [ ]:
fig = CreateFigures.plot_daily_profile(
    df, "Water Heating Electric Power (kW)", plot_max=False, plot_min=False
)

In [ ]:
fig = CreateFigures.plot_time_series_detailed((df["Hot Water Outlet Temperature (C)"],))

## <a name="fleet"></a>Running a fleet


### <a name="ev-fleet"></a>Running an EV fleet

For more details and examples, see
[bin/run_fleet.py](https://github.com/NREL/OCHRE/blob/main/bin/run_fleet.py)

In [ ]:
def setup_ev(i) -> ElectricVehicle:
    # randomly select vehicle type, range, and charging level
    vehicle_type = np.random.choice(["BEV", "PHEV"])
    charging_level = np.random.choice(["Level 1", "Level 2"])
    if vehicle_type == "BEV":
        range = round(np.random.uniform(100, 300))
    else:
        range = round(np.random.uniform(20, 70))

    # Option to specify a file with EV charging events
    # Defaults to older charging event data
    # equipment_event_file = None
    lvl = charging_level.lower().replace(" ", "_")
    equipment_event_file = os.path.join(default_input_path, "EV", f"{vehicle_type}_{lvl}.csv")

    # Initialize equipment
    return ElectricVehicle(
        name=f"EV_{i}",
        seed=i,  # used to randomize charging events. Not used for randomization above
        vehicle_type=vehicle_type,
        charging_level=charging_level,
        range=range,
        start_time=dt.datetime(2018, 1, 1, 0, 0),  # year, month, day, hour, minute
        time_res=dt.timedelta(minutes=15),
        duration=dt.timedelta(days=5),
        verbosity=1,
        save_results=False,  # if True, must specify output_path
        # output_path=os.getcwd(),
        equipment_event_file=equipment_event_file,
    )

# Create fleet
n = 4
fleet = [setup_ev(i + 1) for i in range(n)]


In [ ]:
def run_ev(ev: ElectricVehicle):
    df = ev.simulate()
    out = df["EV Electric Power (kW)"]
    out.name = ev.name
    return out

# Simulate fleet
results = []
for ev in fleet:
    results.append(run_ev(ev))

# combine load profiles
df = pd.concat(results, axis=1)

df.head()


In [ ]:
df.plot()


### <a name="house-fleet"></a>Running multiple houses

For more details and examples, see
[bin/run_multiple.py](https://github.com/NREL/OCHRE/blob/main/bin/run_multiple.py)

In [ ]:
import shutil

from ochre import Analysis

# Note: see documentation for where to download other weather files
# https://ochre-nrel.readthedocs.io/en/latest/InputsAndArguments.html#weather-file
default_weather_file = os.path.join(default_input_path, "Weather", "G0800310.epw")

main_path = os.getcwd()

# Download ResStock files to current directory
buildings = ["bldg0112631"]
upgrades = ["up00", "up11"]
input_paths = []
for upgrade in upgrades:
    for building in buildings:
        input_path = os.path.join(main_path, building, upgrade)
        os.makedirs(input_path, exist_ok=True)
        Analysis.download_resstock_model(building, upgrade, input_path, overwrite=False)
        shutil.copy(default_weather_file, input_path)
        input_paths.append(input_path)


In [ ]:
from ochre.cli import create_dwelling

# Run Dwelling models sequentially
for input_path in input_paths:
    dwelling = create_dwelling(input_path, duration=7)
    dwelling.simulate()


## <a name="control"></a>Running with external controllers

For more details and examples, see
[bin/run_external_control.py](https://github.com/NREL/OCHRE/blob/main/bin/run_external_control.py)

### <a name="hvac-control"></a>HVAC setpoint control

This control will reduce the heating setpoint by 1C from 5-9PM each day.

We use the same house model as above.

In [ ]:
# Initialize
dwelling_args.update(
    {
        "time_res": dt.timedelta(minutes=1),  # time resolution of the simulation
        "duration": dt.timedelta(days=1),  # duration of the simulation
        "verbosity": 6,  # verbosity of time series files (0-9)
    }
)
dwelling = Dwelling(**dwelling_args)

# Get HVAC heater schedule
heater = dwelling.get_equipment_by_end_use("HVAC Heating")
schedule = heater.schedule

# Reduce heating setpoint by 1C from 5-9PM (setpoint is already in the schedule)
peak_times = (schedule.index.hour >= 17) & (schedule.index.hour < 21)
schedule.loc[peak_times, "HVAC Heating Setpoint (C)"] -= 1

# Adjust the HVAC deadband temperature (not in the schedule yet)
schedule["HVAC Heating Deadband (C)"] = 1
schedule.loc[peak_times, "HVAC Heating Deadband (C)"] = 2

# Reset the schedule to implement the changes
heater.reset_time()

# Simulate
df, _, _ = dwelling.simulate()


In [ ]:
cols_to_plot = [
    "HVAC Heating Setpoint (C)",
    "Temperature - Indoor (C)",
    "Temperature - Outdoor (C)",
    "Unmet HVAC Load (C)",
    "HVAC Heating Electric Power (kW)",
]
df.loc[:, cols_to_plot].plot()


### <a name="ev-control"></a>EV managed charging


In [ ]:
from ochre.Equipment.EV import EV_EFFICIENCY

equipment_args = {
    "start_time": dt.datetime(2018, 1, 1, 0, 0),  # year, month, day, hour, minute
    "time_res": dt.timedelta(minutes=60),
    "duration": dt.timedelta(days=20),
    "verbosity": 3,
    "save_results": False,  # if True, must specify output_path
    # "output_path": os.getcwd(),
    # Equipment parameters
    "vehicle_type": "BEV",
    "charging_level": "Level 1",
    "range": 150,
}

# Initialize
ev = ElectricVehicle(**equipment_args)

# slow charge from start to end of parking
for t in ev.sim_times:
    remaining_hours = (ev.event_end - t).total_seconds() / 3600
    remaining_kwh = (1 - ev.soc) * ev.capacity
    if t >= ev.event_start and remaining_hours:
        power = remaining_kwh / remaining_hours / EV_EFFICIENCY
        ev.update({"Max Power": power})
    else:
        ev.update()

df = ev.finalize()


In [ ]:
CreateFigures.plot_daily_profile(df, "EV Electric Power (kW)", plot_max=False, plot_min=False)


In [ ]:
df.loc[:, ["EV Electric Power (kW)", "EV Unmet Load (kWh)", "EV SOC (-)"]].plot()


### <a name="wh-control"></a>HPWH CTA-2045 control


In [ ]:
from ochre import HeatPumpWaterHeater

# Define equipment and simulation parameters
setpoint_default = 51  # in C
deadband_default = 5.56  # in C
equipment_args = {
    "start_time": dt.datetime(2018, 1, 1, 0, 0),  # year, month, day, hour, minute
    "time_res": dt.timedelta(minutes=1),
    "duration": dt.timedelta(days=1),
    "verbosity": 6,  # required to get setpoint and deadband in results
    "save_results": False,  # if True, must specify output_path
    # "output_path": os.getcwd(),        # Equipment parameters
    "Setpoint Temperature (C)": setpoint_default,
    "Tank Volume (L)": 250,
    "Tank Height (m)": 1.22,
    "UA (W/K)": 2.17,
    "HPWH COP (-)": 4.5,
}

# Create water draw schedule
times = pd.date_range(
    equipment_args["start_time"],
    equipment_args["start_time"] + equipment_args["duration"],
    freq=equipment_args["time_res"],
    inclusive="left",
)
water_draw_magnitude = 12  # L/min
withdraw_rate = np.random.choice([0, water_draw_magnitude], p=[0.99, 0.01], size=len(times))
schedule = pd.DataFrame(
    {
        "Water Heating (L/min)": withdraw_rate,
        "Water Heating Setpoint (C)": setpoint_default,  # Setting so that it can reset
        "Water Heating Deadband (C)": deadband_default,  # Setting so that it can reset
        "Zone Temperature (C)": 20,
        "Zone Wet Bulb Temperature (C)": 15,  # Required for HPWH
        "Mains Temperature (C)": 7,
    },
    index=times,
)

# Initialize equipment
hpwh = HeatPumpWaterHeater(schedule=schedule, **equipment_args)

# Simulate
control_signal = {}
for t in hpwh.sim_times:
    # Change setpoint based on hour of day
    if t.hour in [7, 16]:
        # CTA-2045 Basic Load Add command
        control_signal = {"Deadband": deadband_default - 2.78}
    elif t.hour in [8, 17]:
        # CTA-2045 Load Shed command
        control_signal = {
            "Setpoint": setpoint_default - 5.56,
            "Deadband": deadband_default - 2.78,
        }
    else:
        control_signal = {}

    # Run with controls
    _ = hpwh.update(control_signal=control_signal)

df = hpwh.finalize()

df.head()


In [ ]:
cols_to_plot = [
    "Hot Water Outlet Temperature (C)",
    "Hot Water Average Temperature (C)",
    "Water Heating Deadband Upper Limit (C)",
    "Water Heating Deadband Lower Limit (C)",
    "Water Heating Electric Power (kW)",
    "Hot Water Unmet Demand (kW)",
    "Hot Water Delivered (L/min)",
]
df.loc[:, cols_to_plot].plot()


## <a name="links"></a>Links to other examples

For more examples, see Python example scripts to:
  * Run a [single dwelling](https://github.com/NREL/OCHRE/blob/main/bin/run_dwelling.py)
  * Run a [single piece of equipment](https://github.com/NREL/OCHRE/blob/main/bin/run_equipment.py)
  * Run a [fleet of equipment](https://github.com/NREL/OCHRE/blob/main/bin/run_fleet.py)
  * Run [multiple dwellings](https://github.com/NREL/OCHRE/blob/main/bin/run_multiple.py)
  * Run a [OCHRE with an external controller](https://github.com/NREL/OCHRE/blob/main/bin/run_external_control.py)
  * Run a [OCHRE in co-simulation using HELICS](https://github.com/NREL/OCHRE/blob/main/bin/run_cosimulation.py)

More information about usage can be found in the [OCHRE documentation](https://ochre-nrel.readthedocs.io/en/latest/)
